In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
from fastai.tabular import *
!pip install category_encoders
from IPython import display
display.clear_output()

In [0]:
df = pd.read_csv('doctor_train.csv')
tdf = pd.read_csv('doctor_test.csv')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [12]:
# df.info()
def dataessing(df, test):
  cat_col = ['Profession','Status','edu','Irregular','residence','prev_diagnosed','communication','Month','side_effects']
  num_col = ['age','day','Time','Doctor_visits','last_visit','cured_in']
  loan = ['Money']
  target = 'Y'
  train_DF = pd.DataFrame(); valid_DF = pd.DataFrame(); test_DF = pd.DataFrame();
  dfs = [df, test]
  for i in range(0,2):
    d = dfs[i]
    dfc = pd.DataFrame()
    dfc[cat_col] = d[cat_col].fillna("unknown")
    dfc[num_col+loan] = d[num_col+loan].fillna(0)
    dfc[num_col] = dfc[num_col].clip(lower=0)
    dfcd = dfc
    if i == 0:
      from sklearn.model_selection import train_test_split
      train_DF, valid_DF, y_train, y_test = train_test_split(dfcd, df[target], random_state=45, test_size=0.3)
      train_DF['Y'] = y_train
      valid_DF['Y'] = y_test
    else:
      test_DF = dfcd
  return train_DF, valid_DF, test_DF, num_col, cat_col, target

train_DF, valid_DF, test_DF, num_col, cat_col, target = dataessing(df, tdf)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
import category_encoders as ce
cb_enc = ce.CatBoostEncoder(cols=cat_col, random_state=7)
# Learn encoding from the training set
cb_enc.fit(train_DF[cat_col], train_DF['Y'])
# Apply encoding to the train and validation sets
train_DF[cat_col] = cb_enc.transform(train_DF[cat_col])
valid_DF[cat_col] = cb_enc.transform(valid_DF[cat_col])

In [49]:
feature_cols = train_DF.columns.drop('Y')
from sklearn import metrics
import lightgbm as lgb
dtrain = lgb.Dataset(train_DF[feature_cols], label=train_DF['Y'])
dvalid = lgb.Dataset(valid_DF[feature_cols], label=valid_DF['Y'])
param = {'num_leaves': 64, 'objective': 'binary', 'metric': 'auc', 'seed': 7}
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=20, verbose_eval=False)
valid_pred = bst.predict(valid_DF[feature_cols])
valid_pred=valid_pred.round(0)
valid_score = metrics.roc_auc_score(valid_DF['Y'], valid_pred)
print(f"Validation AUC score: {valid_score}")
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
print(accuracy_score(valid_DF['Y'], valid_pred))
print(confusion_matrix(valid_DF['Y'], valid_pred))
print(classification_report(valid_DF['Y'], valid_pred))
print(roc_auc_score(valid_DF['Y'], valid_pred))

Validation AUC score: 0.7152317629640417
0.9058990424400047
[[7210  281]
 [ 515  453]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      7491
           1       0.62      0.47      0.53       968

    accuracy                           0.91      8459
   macro avg       0.78      0.72      0.74      8459
weighted avg       0.90      0.91      0.90      8459

0.7152317629640417


In [0]:
test_DF[cat_col] = cb_enc.transform(test_DF[cat_col])
valid_pred = bst.predict(test_DF[feature_cols])
valid_pred=valid_pred.round(0).astype(int)
le_y = le.inverse_transform(valid_pred)
output = pd.DataFrame({'id': tdf.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')

# final pred code!